In [1]:
from dotenv import load_dotenv
from typing import Annotated,TypedDict
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
import os
from langgraph.graph.message import add_messages
from pydantic import BaseModel,Field

load_dotenv()

/Users/divyyadav/miniforge3/envs/new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
import datetime
from typing import Optional

def now_iso():
    """Return current datetime in ISO format including time and timezone if available."""
    return datetime.datetime.now(datetime.timezone.utc).isoformat()

class AnalysisSource(BaseModel):
    source_name:  Optional[str] = Field(..., description="Name of the data source, e.g., Google, Bing, Reddit, Google Finance")
    analysis: str = Field(..., description="Analysis result or summary from this source")
    source_link: Optional[str] = Field(..., description="Direct link to the information or post")


class LLMAnalysisResult(BaseModel):
    user_question: str = Field(..., description="The user's original question")
    sources: list[AnalysisSource] = Field(..., description="List of sources with analysis and URLs")
    synthesized_answer: str = Field(..., description="LLM's synthesized answer using all sources")

In [3]:
llm=ChatGoogleGenerativeAI(model='gemini-2.5-flash',api_key=os.getenv('GEMINI_API_KEY'),async_client=True)
analysing_llm=llm.with_structured_output(AnalysisSource)

In [4]:
await llm.ainvoke('give me aa para about ethics in ai')

AIMessage(content='Ethics in AI is a critical and rapidly evolving field focused on ensuring that artificial intelligence systems are developed, deployed, and used in a manner that aligns with human values, societal well-being, and moral principles. It grapples with profound questions like algorithmic bias, which can perpetuate and amplify existing societal inequalities; the imperative for transparency and explainability in AI decision-making; and the complex issues of accountability when AI systems make errors or cause harm. Addressing these ethical dilemmas is paramount to fostering public trust, preventing unintended negative consequences, protecting individual privacy and autonomy, and ultimately ensuring that AI serves as a beneficial tool that enhances, rather than diminishes, human dignity and fairness across all domains.', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-98302b1d-b01c-40b0-8

In [5]:
content="The rapid advancement of Artificial Intelligence inextricably links its development with profound ethical considerations. Ensuring fairness and mitigating algorithmic bias, often stemming from unrepresentative training data, is paramount to prevent discrimination and perpetuate societal inequalities. Equally crucial are establishing clear lines of accountability for AI's actions, promoting transparency in its decision-making processes, and safeguarding user privacy in data collection and utilization. Furthermore, the ethical deployment of AI demands careful consideration of its societal impact, from potential job displacement to the need for maintaining human oversight over autonomous systems, preventing misuse that could undermine democratic processes or exacerbate social inequalities. Ultimately, integrating ethical principles into AI's design, deployment, and governance is not merely a philosophical exercise, but a critical imperative to ensure this transformative technology serves humanity beneficially and equitably."

In [6]:
await analysing_llm.ainvoke(content)

## Define state


In [7]:
class ResearchState(TypedDict):
    messages:Annotated[str,add_messages]
    user_question:str|None
    google_search_results:str|None
    google_finance_results:str|None
    bing_search_results:str|None
    reddit_search_results:str|None
    selected_reddit_urls:list[str]|None
    reddit_posts:str|None
    google_analysis:str
    bing_analysis:str
    reddit_analysis:str
    google_finance_analysis:str
    synthesized_answer:str

# NODES

In [8]:
from ast import Return


def google_search(state:ResearchState)->ResearchState:
    return state


def bing_search(state:ResearchState)->ResearchState:
    return state


def reddit_search(state:ResearchState)->ResearchState:
    return state

def twitter_search(state: ResearchState) -> ResearchState:
    return state

def google_finance_search(state:ResearchState)->ResearchState:
    return state

def analysis_google_results(state:ResearchState)->ResearchState:
    return state


def analysis_bing_results(state:ResearchState)->ResearchState:
    return state


def analysis_reddit_results(state:ResearchState)->ResearchState:
    return state


def analyze_results_results(state:ResearchState)->ResearchState:
    return state
    

def synthesize_results(state:ResearchState)->ResearchState:
    return state


def final_results(state:ResearchState)->ResearchState:
    return state

# GRAPH

In [9]:
graph=StateGraph(ResearchState)

In [ ]:
graph.add_node("google_search", google_search)
graph.add_node("bing_search", bing_search)
graph.add_node("reddit_search", reddit_search)
graph.add_node("analysis_google_results", analysis_google_results)
graph.add_node("analysis_bing_results", analysis_bing_results)
graph.add_node("analysis_reddit_results", analysis_reddit_results)
graph.add_node("analyze_results_results", analyze_results_results)
graph.add_node("synthesize_results", synthesize_results)
graph.add_node("final_results", final_results)

In [ ]:
graph.add_edge(START, "google_search")
graph.add_edge(START, "bing_search")
graph.add_edge(START, "reddit_search")

In [45]:
import asyncpraw
import aiocache
from aiocache import Cache,caches

client_id = '2ns033kTrWIIpDJSBDJZYw'
client_secret = '0Jm2zTDhaBV6VNbGhyg0PPTnlnD-SQ'
user_agent = 'dp by /u/Temporary_Version105 '

# --- Configure aiocache ---
# We'll use a simple file-based cache stored in a folder named './cache_storage'
# It will use JSON to store the data (serializer)
caches.set_config({
    'default': {
        'cache': "aiocache.cache.FileCache",
        'cache_directory': "./cache_storage",
        'serializer': {
            'class': "aiocache.serializers.JsonSerializer"
        }
    }
})
async def praw():
    """The main async function, now using aiocache."""
    cache=caches.get('default')
    
    print("Connecting to Reddit...")
    reddit = asyncpraw.Reddit(
        client_id=client_id,
        client_secret=client_secret,
        user_agent=user_agent
    )

    subreddit = await reddit.subreddit('finance')
    
    post_processed=0

    async for post in subreddit.hot(limit=50):
        post_processed+=1

        cached_data=await caches.get(post.id)

        if cached_data:
        
            print(f"  > Using cached data for post: {cached_data['title']}")

        else:
            comments=[]
            await post.load()

            for comment in post.comments.list()[:10]:
                if hasattr(comment,'body'):
                       comments.append(comment.body)

            print(post_processed)
            post_data = {"title": post.title, "score": post.score, "comments": comments}

            await cache.set(post.id, post_data, ttl=3600)
        
    await cache.close()
        
            

In [46]:
praw=await praw()


ModuleNotFoundError: No module named 'aiocache.cache'

In [34]:
pip install "aiocache[file]"

Note: you may need to restart the kernel to use updated packages.


# FUNCTIONS

In [27]:
a[::-1]
    

'eb ih'

In [13]:
for key,val in cache.items():
    print(key)

1oh6tsf
1oevvop
1obb89t
1o87gos
1o7dul7
1o5k55a
1o5as1r
1o2swix
1o2n7yn
1o2h0ih
